# Wound segmentation using Double U-Net

This notebook contains all the code necessary to train the Double U-Net to segment wounds in the Redscar Dataset.

## Dependencies

In [ ]:
from matplotlib import pyplot as plt
from numpy.random import seed
from os import listdir
from os.path import isfile, join
from sklearn.metrics import jaccard_score

import cv2
import glob
import json
import numpy as np
import os
import pandas
import re
import skimage
import skimage.color
import skimage.io
import skimage.transform
import tensorflow as tf
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
tf.compat.v1.disable_eager_execution()

from typing import Tuple

# This is a custom library that includes the different architectures considered in this study.
from cnn_architectures.architectures.models.double_unet.double_unet import DoubleUNet
# To install it, you have to grab the 1.0 version uploaded on the GitHub repository. 
# Important: There are other versions of the library, but these architectures are implemented and trained with 1.0.
# pip install git+https://github.com/mmunar97/cnn_architectures

seed(1)

## Previous considerations

### Declaration of global variables

In [ ]:
REDSCAR_DATASET_PATH = r"/home/marc/UIB_EXPERIMENTS/REDSCAR"
REDSCAR_ML_PATH = os.path.join(REDSCAR_DATASET_PATH, r"SUBSETS/MACHINE_LEARNING_DATASET")
REDSCAR_ML_TRAIN_PATH = os.path.join(REDSCAR_ML_PATH, r"train")
REDSCAR_ML_TEST_PATH  = os.path.join(REDSCAR_ML_PATH, r"test")

### Declaration of a Keras generator

In [ ]:
def wound_generator(img_path: str, gt_path: str, size, batch_size: int):
    images = sorted(glob.glob(img_path))
    masks = sorted(glob.glob(gt_path))
    
    batch_img = []
    batch_mask = []
    idx = 0
    while(True):
        path_img = images[idx % len(images)]
        path_mask = masks[idx % len(images)]
    
        img = cv2.imread(path_img, 1)        
        mask = cv2.imread(path_mask, 0)
        
        img = skimage.transform.resize(img, (size[0], size[1], 3))
        mask = cv2.resize(mask, size)
        
        mask = mask / 255

        batch_img.append(img)
        batch_mask.append(np.dstack((mask, mask)))
        
        if ((idx % batch_size - 1) == 0) and idx != 0:            
            batch_img = np.array(batch_img)
            batch_mask = np.array(batch_mask)
            
            yield batch_img, batch_mask
            
            batch_img = []
            batch_mask = []
        idx += 1

In [ ]:
for image, mask in wound_generator(REDSCAR_ML_TRAIN_PATH+'/IMAGES/*.png', REDSCAR_ML_TRAIN_PATH+'/GT_WOUND_MASK/*.png', (512, 512), 5):
    print(image.shape, mask.shape)
    plt.imshow(image[1,:,:])
    plt.show()
    plt.imshow(mask[1,:,:,0])
    plt.show()
    break

### Implementation of metrics for the training process

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras import backend 

def dice(y_true, y_pred):
    y_true_f = backend.flatten(y_true)
    y_pred_f = backend.flatten(y_pred)
    intersection = backend.sum(y_true_f * y_pred_f)
    return (2. * intersection+1 ) / (backend.sum(y_true_f) + backend.sum(y_pred_f)+1 )

def dice_loss(y_true, y_pred):
    return 1-dice(y_true, y_pred)

reduceLROnPlat = ReduceLROnPlateau(monitor='dice', factor=0.5,
                                   patience=3,
                                   verbose=1, mode='max', cooldown=2, min_lr=1e-7)

early = EarlyStopping(monitor="dice",
                      mode="max",
                      patience=8) 
callbacks_list = [early, reduceLROnPlat]

## Training of the different models

### Double U-Net with 25 epochs

We initialise the model by adjusting some parameters.

In [ ]:
################## PARAMETERS ##################
BATCH_SIZE = 5
TOTAL_IMAGES = 275
STEPS_PER_EPOCH = TOTAL_IMAGES // BATCH_SIZE 
EPOCHS = 25
INPUT_SIZE = 256

EXPERIMENT_PATH = r"/home/marc/UIB_EXPERIMENTS/CNN_EXPERIMENTS/DoubleUNet/EXPERIMENT1"
################################################

model1 = DoubleUNet(input_size=(INPUT_SIZE,INPUT_SIZE, 3))

model1.build()
model1.compile(loss_func=[dice_loss], metrics = [dice])

We check that the network structure is as desired.

In [ ]:
model1.model.summary()

We proceed with the training.

In [ ]:
model1.train(train_generator=wound_generator(REDSCAR_ML_TRAIN_PATH+'/IMAGES/*.png', REDSCAR_ML_TRAIN_PATH+'/GT_WOUND_MASK/*.png', (INPUT_SIZE, INPUT_SIZE), BATCH_SIZE), 
             val_generator=wound_generator(REDSCAR_ML_TEST_PATH+'/IMAGES/*.png', REDSCAR_ML_TEST_PATH+'/GT_WOUND_MASK/*.png', (INPUT_SIZE, INPUT_SIZE), BATCH_SIZE), 
             epochs=EPOCHS, steps_per_epoch=STEPS_PER_EPOCH, 
             check_point_path=None, validation_steps=2)

Once the training is finished, we save the model for future predictions.

In [ ]:
model1.model.save_weights(EXPERIMENT_PATH+f'/dun1_epochs={EPOCHS}_lr=3e-5_res=0.h5')

Let's look at how the training has performed during the different periods, as well as the performance of that training on the test set.

In [ ]:
history = model1.history

plt.figure(figsize=(9,6), dpi= 100, facecolor='w', edgecolor='k')

plt.subplot(1,2,1)
plt.plot(history.history['dice'])
plt.plot(history.history['val_dice'])
plt.title('Dice evolution')
plt.ylabel('dice')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')

plt.subplot(1,2,2)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')

plt.savefig(EXPERIMENT_PATH+f'/dun1.png')

### Double U-Net with 55 epochs

We initialise the model by adjusting some parameters.

In [ ]:
################## PARAMETERS ##################
BATCH_SIZE = 5
TOTAL_IMAGES = 275
STEPS_PER_EPOCH = TOTAL_IMAGES // BATCH_SIZE 
EPOCHS = 55
INPUT_SIZE = 256

EXPERIMENT_PATH = r"/home/marc/UIB_EXPERIMENTS/CNN_EXPERIMENTS/DoubleUNet/EXPERIMENT1"
################################################

model2 = DoubleUNet(input_size=(INPUT_SIZE,INPUT_SIZE, 3))

model2.build()
model2.compile(loss_func=[dice_loss], metrics = [dice])

We check that the network structure is as desired.

In [ ]:
model2.model.summary()

We proceed with the training.

In [ ]:
model2.train(train_generator=wound_generator(REDSCAR_ML_TRAIN_PATH+'/IMAGES/*.png', REDSCAR_ML_TRAIN_PATH+'/GT_WOUND_MASK/*.png', (INPUT_SIZE, INPUT_SIZE), BATCH_SIZE), 
             val_generator=wound_generator(REDSCAR_ML_TEST_PATH+'/IMAGES/*.png', REDSCAR_ML_TEST_PATH+'/GT_WOUND_MASK/*.png', (INPUT_SIZE, INPUT_SIZE), BATCH_SIZE), 
             epochs=EPOCHS, steps_per_epoch=STEPS_PER_EPOCH, 
             check_point_path=None, validation_steps=2)

Once the training is finished, we save the model for future predictions.

In [ ]:
model2.model.save_weights(EXPERIMENT_PATH+f'/dun2_epochs={EPOCHS}_lr=3e-5_res=0.h5')

Let's look at how the training has performed during the different periods, as well as the performance of that training on the test set.

In [ ]:
history = model2.history

plt.figure(figsize=(9,6), dpi= 100, facecolor='w', edgecolor='k')

plt.subplot(1,2,1)
plt.plot(history.history['dice'])
plt.plot(history.history['val_dice'])
plt.title('Dice evolution')
plt.ylabel('dice')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')

plt.subplot(1,2,2)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')

plt.savefig(EXPERIMENT_PATH+f'/dun2.png')

### Double U-Net with 100 epochs

We initialise the model by adjusting some parameters.

In [ ]:
################## PARAMETERS ##################
BATCH_SIZE = 5
TOTAL_IMAGES = 275
STEPS_PER_EPOCH = TOTAL_IMAGES // BATCH_SIZE 
EPOCHS = 100
INPUT_SIZE = 256

EXPERIMENT_PATH = r"/home/marc/UIB_EXPERIMENTS/CNN_EXPERIMENTS/DoubleUNet/EXPERIMENT1"
################################################

model3 = DoubleUNet(input_size=(INPUT_SIZE,INPUT_SIZE, 3))

model3.build()
model3.compile(loss_func=[dice_loss], metrics = [dice])

We check that the network structure is as desired.

In [ ]:
model3.model.summary()

We proceed with the training.

In [ ]:
model3.train(train_generator=wound_generator(REDSCAR_ML_TRAIN_PATH+'/IMAGES/*.png', REDSCAR_ML_TRAIN_PATH+'/GT_WOUND_MASK/*.png', (INPUT_SIZE, INPUT_SIZE), BATCH_SIZE), 
             val_generator=wound_generator(REDSCAR_ML_TEST_PATH+'/IMAGES/*.png', REDSCAR_ML_TEST_PATH+'/GT_WOUND_MASK/*.png', (INPUT_SIZE, INPUT_SIZE), BATCH_SIZE), 
             epochs=EPOCHS, steps_per_epoch=STEPS_PER_EPOCH, 
             check_point_path=None, validation_steps=2)

Once the training is finished, we save the model for future predictions.

In [ ]:
model3.model.save_weights(EXPERIMENT_PATH+f'/dun3_epochs={EPOCHS}_lr=3e-5_res=0.h5')

Let's look at how the training has performed during the different periods, as well as the performance of that training on the test set.

In [ ]:
history = model3.history

plt.figure(figsize=(9,6), dpi= 100, facecolor='w', edgecolor='k')

plt.subplot(1,2,1)
plt.plot(history.history['dice'])
plt.plot(history.history['val_dice'])
plt.title('Dice evolution')
plt.ylabel('dice')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')

plt.subplot(1,2,2)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')

plt.savefig(EXPERIMENT_PATH+f'/dun3.png')

## Evaluation of models on the test set

Once the different models have been trained with the train set, we proceed to evaluate them on the test set. First, we declare the following function that allows us to evaluate the whole test set on a certain trained model.

In [ ]:
def predict_from_model(model: DoubleUNet, 
                       image: np.ndarray, 
                       binary_threshold: float):
    resized_image_normalized = skimage.transform.resize(image, (model.input_size[0], model.input_size[1], 3))
    prediction = model.model.predict(np.array([resized_image_normalized]))
    raw_prediction = prediction[0][:, :, 1]
    raw_prediction_resized = skimage.transform.resize(raw_prediction, (image.shape[0], image.shape[1]))
        
    prediction_mask_bool = raw_prediction_resized >= binary_threshold
    prediction_mask_int = 255 * prediction_mask_bool
    
    colormask = image.copy()
    colormask[:, :, 0][prediction_mask_bool] = 0
    colormask[:, :, 1][prediction_mask_bool] = 255
    colormask[:, :, 2][prediction_mask_bool] = 0
        
    return prediction_mask_bool, prediction_mask_int, colormask


def compute_metrics(model: DoubleUNet, 
                    images_path: str, 
                    gt_path: str, 
                    binary_threshold: float, 
                    prediction_saving_path: str, 
                    measures_saving_path: str, 
                    measures_file_name: str):
    
    files = []
    jaccard_values = []
    
    for file in listdir(images_path):
        if isfile(join(images_path, file)):
            
            filename = file.replace(".png", "")
            
            gt_image = cv2.imread(gt_path+rf"/{file}", cv2.IMREAD_GRAYSCALE)
            gt_mask = gt_image/255
            gt_mask = gt_mask.astype(int)
            
            image = cv2.imread(images_path+rf"/{file}")
            
            prediction_mask_bool, prediction_mask_int, prediction_colormask = predict_from_model(model=model, 
                                                                                                 image=image, 
                                                                                                 binary_threshold=binary_threshold)
            
            cv2.imwrite(prediction_saving_path+rf"/{filename}_colormask.png", prediction_colormask)
            cv2.imwrite(prediction_saving_path+rf"/{filename}_mask.png", prediction_mask_int)

            jaccard_value = jaccard_score(gt_mask.flatten(), prediction_mask_bool.flatten())
            files.append(file)
            jaccard_values.append(round(jaccard_value, 6))
    
    data = pandas.DataFrame({
        'IMAGES': files, 'JACCARD': jaccard_values
    })
    data.to_csv(measures_saving_path+rf'/{measures_file_name}.txt', index=None)

### Evaluation of Double U-Net trained with 25 epochs

In [ ]:
# Optional. Uncomment the following lines if the model is already trained and you want to load it from disk.

# EXPERIMENT_PATH = r"/home/marc/UIB_EXPERIMENTS/CNN_EXPERIMENTS/DoubleUNet/EXPERIMENT1"
# model1_path = "/home/marc/UIB_EXPERIMENTS/CNN_EXPERIMENTS/DoubleUNet/EXPERIMENT1/dun1_epochs=25_lr=3e-5_res=0.h5"
# model1 = DoubleUNet(input_size=(256, 256, 3))

# model1.build()
# model1.load_weight(model1_path)

In [ ]:
compute_metrics(model=model1, images_path=REDSCAR_ML_TRAIN_PATH+'/IMAGES', gt_path=REDSCAR_ML_TRAIN_PATH+'/GT_WOUND_MASK', binary_threshold=0.5, 
                prediction_saving_path=EXPERIMENT_PATH+'/dun1_evaluation/train', 
                measures_saving_path=EXPERIMENT_PATH+'/dun1_evaluation', 
                measures_file_name='train')

compute_metrics(model=model1, images_path=REDSCAR_ML_TEST_PATH+'/IMAGES', gt_path=REDSCAR_ML_TEST_PATH+'/GT_WOUND_MASK', binary_threshold=0.5, 
                prediction_saving_path=EXPERIMENT_PATH+'/dun1_evaluation/test', 
                measures_saving_path=EXPERIMENT_PATH+'/dun1_evaluation', 
                measures_file_name='test')

### Evaluation of Double U-Net trained with 55 epochs

In [ ]:
# Optional. Uncomment the following lines if the model is already trained and you want to load it from disk.

# EXPERIMENT_PATH = r"/home/marc/UIB_EXPERIMENTS/CNN_EXPERIMENTS/DoubleUNet/EXPERIMENT1"
# model2_path = "/home/marc/UIB_EXPERIMENTS/CNN_EXPERIMENTS/DoubleUNet/EXPERIMENT1/dun2_epochs=55_lr=3e-5_res=0.h5"
# model2 = DoubleUNet(input_size=(256, 256, 3))

# model2.build()
# model2.load_weight(model2_path)

In [ ]:
compute_metrics(model=model2, images_path=REDSCAR_ML_TRAIN_PATH+'/IMAGES', gt_path=REDSCAR_ML_TRAIN_PATH+'/GT_WOUND_MASK', binary_threshold=0.5, 
                prediction_saving_path=EXPERIMENT_PATH+'/dun2_evaluation/train', 
                measures_saving_path=EXPERIMENT_PATH+'/dun2_evaluation', 
                measures_file_name='train')

compute_metrics(model=model2, images_path=REDSCAR_ML_TEST_PATH+'/IMAGES', gt_path=REDSCAR_ML_TEST_PATH+'/GT_WOUND_MASK', binary_threshold=0.5, 
                prediction_saving_path=EXPERIMENT_PATH+'/dun2_evaluation/test', 
                measures_saving_path=EXPERIMENT_PATH+'/dun2_evaluation', 
                measures_file_name='test')

### Evaluation of Double U-Net trained with 100 epochs

In [ ]:
# Optional. Uncomment the following lines if the model is already trained and you want to load it from disk.

# EXPERIMENT_PATH = r"/home/marc/UIB_EXPERIMENTS/CNN_EXPERIMENTS/DoubleUNet/EXPERIMENT1"
# model3_path = "/home/marc/UIB_EXPERIMENTS/CNN_EXPERIMENTS/DoubleUNet/EXPERIMENT1/dun3_epochs=100_lr=3e-5_res=0.h5"
# model3 = DoubleUNet(input_size=(256, 256, 3))

# model3.build()
# model3.load_weight(model3_path)

In [ ]:
compute_metrics(model=model3, images_path=REDSCAR_ML_TRAIN_PATH+'/IMAGES', gt_path=REDSCAR_ML_TRAIN_PATH+'/GT_WOUND_MASK', binary_threshold=0.5, 
                prediction_saving_path=EXPERIMENT_PATH+'/dun3_evaluation/train', 
                measures_saving_path=EXPERIMENT_PATH+'/dun3_evaluation', 
                measures_file_name='train')

compute_metrics(model=model3, images_path=REDSCAR_ML_TEST_PATH+'/IMAGES', gt_path=REDSCAR_ML_TEST_PATH+'/GT_WOUND_MASK', binary_threshold=0.5, 
                prediction_saving_path=EXPERIMENT_PATH+'/dun3_evaluation/test', 
                measures_saving_path=EXPERIMENT_PATH+'/dun3_evaluation', 
                measures_file_name='test')